In [0]:
import pandas as pd
import numpy as np
import torch
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class ECG_dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.ecg_frame = pd.read_csv(csv_file)
        self.ecg_frame = self.ecg_frame.drop(columns = ['Unnamed: 0'])
        self.transform = transform
        le = LabelEncoder()
        self.ecg_frame.Label = le.fit_transform(self.ecg_frame.Label)

    def __len__(self):
        return len(self.ecg_frame)
    
    def __getitem__(self, idx):
      if torch.is_tensor(idx):
        idx = idx.tolist()
      signal = np.asarray(self.ecg_frame.iloc[idx, :-1].values).reshape(1,1000).astype('float16')
      label = self.ecg_frame.Label.values[idx]    
      sample = {'signal': signal, 'label': label}
      if self.transform:
          signal_as_tensor = self.transform(signal)[0][0]
      return (signal_as_tensor, label)

In [0]:
csv = '/content/gdrive/My Drive/ECG_data/test_1000.csv'
transformations = transforms.Compose([transforms.ToTensor()])

ECG_data = ECG_dataset(csv_file = csv, transform = transformations)
dataloader = DataLoader(ECG_data, batch_size=4,
                        shuffle=True, num_workers=4)

In [111]:
for i,batch in enumerate(dataloader):
    print(i, batch)
    break

0 [tensor([[-0.0050,  0.0250,  0.0150,  ..., -0.0150,  0.0050, -0.0050],
        [ 0.0751,  0.0909,  0.0815,  ...,  0.0523,  0.0695,  0.0622],
        [ 0.1411,  0.1509,  0.1376,  ...,  0.0120,  0.0092,  0.0666],
        [-0.4438, -0.4265, -0.4170,  ..., -0.4985, -0.4919, -0.4893]],
       dtype=torch.float16), tensor([2, 0, 0, 0])]
